## Single-parameter optimization with BoTorch
This notebook illustrates how to minimize the shadow factor using BoTorch.

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf


In [ ]:
def run_simulation(o2_size: float) -> torch.Tensor:
    # Placeholder simulation returning a very small shadow factor
    noise = 1e-11 * torch.randn(1)
    shadow = (o2_size - 0.5)**2 + 1e-10 + noise
    return shadow


In [ ]:
torch.manual_seed(0)
train_X = torch.rand(5, 1, dtype=torch.double)
train_Y = torch.stack([run_simulation(x.item()) for x in train_X]).unsqueeze(-1)


In [ ]:
bounds = torch.stack([torch.zeros(1), torch.ones(1)]).double()
for i in range(5):
    gp = SingleTaskGP(train_X, -train_Y, input_transform=Normalize(d=1), outcome_transform=Standardize(m=1))
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    acq = LogExpectedImprovement(gp, best_f=(-train_Y).max())
    candidate, _ = optimize_acqf(acq, bounds=bounds, q=1, num_restarts=5, raw_samples=20)
    new_y = run_simulation(candidate.item())
    train_X = torch.cat([train_X, candidate])
    train_Y = torch.cat([train_Y, new_y.unsqueeze(-1)])


In [ ]:
best_value, best_idx = train_Y.min(dim=0)
print(f'Best aperture size: {train_X[best_idx].item():.4f}, shadow factor: {best_value.item():.2e}')
